# User Satsfaction Score Prediction


In [1]:
import pandas as pd
import os
import sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [2]:
# Load the cleaned xdr data into a DataFrame
data = pd.read_csv('../data/cleaned_data.csv', index_col=0)

In [3]:
# Import modules
from user_engagement_analysis import UserEngagementAnalysis
from experience_analysis import ExperienceAnalytics
from satisfaction_analysis import UserSatisfactionAnalytics

In [4]:
# Create instances of engagement and user experience classes
engagement_analysis = UserEngagementAnalysis(data)
experience_analysis = ExperienceAnalytics(data)
satisfaction_analysis = UserSatisfactionAnalytics()

In [5]:
# Normalize and cluster
# Aggregate engagement metrics
engagement_analysis.aggregate_metrics()
engagement_analysis.metrics.head()
engagement_analysis.normalize_and_cluster(n_clusters=3)